## Data gathering - Negative Class only

In [1]:
from datetime import datetime, timedelta  
import pandas as pd
# wildfire_data = pd.read_csv('AK_Fire_Location_cleaned_v2.csv', usecols=['LATITUDE', 'LONGITUDE', 'DISCOVERYD', 'DISCOVERYS', 'ESTIMATEDT'])
# wildfire_data = wildfire_data.dropna()
# wildfire_data = wildfire_data[wildfire_data['LATITUDE']>=64]
# wildfire_data = wildfire_data[pd.to_datetime(wildfire_data['DISCOVERYD']).dt.year<=2015]
# wildfire_data

wildfire_data = pd.read_csv('Task_FireDetection_noRS_negClass_V4.csv')
wildfire_data

,ClusterID,DetectionDate_first,DetectionDate_last,Duration,avgLatitude,avgLongitude,totalArea_km2,maxBrightness,maxBright_t31,maxFRP,avgConfidence
0,2132,2000-28-12,2000-28-12,0,63.098125,-162.816950,0.979713,0,0,0,0
1,2133,2001-19-05,2001-19-05,0,64.926200,-147.158700,0.295989,0,0,0,0
2,2134,2001-16-07,2001-16-07,0,64.504700,-148.539500,0.250000,0,0,0,0
3,2135,2001-28-09,2001-28-09,0,64.133300,-146.508500,0.250000,0,0,0,0
4,2136,2002-19-05,2002-19-05,0,64.164200,-146.352200,0.250000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2126,4258,2022-29-08,2022-29-08,0,55.558100,-161.893500,0.250000,0,0,0,0
2127,4259,2022-30-09,2022-30-09,0,55.562800,-161.890900,0.250000,0,0,0,0
2128,4260,2022-30-09,2022-30-09,0,55.560600,-161.875700,0.250000,0,0,0,0
2129,4261,2022-15-11,2022-17-11,0,55.558150,-161.887600,0.156672,0,0,0,0


In [4]:
import ee
import pandas as pd

# Trigger the authentication flow.
ee.Authenticate()


Enter verification code: 4/1AeaYSHDV6BkRyNxOyY6eUk8PT-_b0wwJWZg9osmV2ZYoikVnUIGmVRmGu5A

Successfully saved authorization token.


In [9]:
import ee
import pandas as pd

ee.Initialize()  # Initialize the Earth Engine module.

def get_time_series(collection, start_date, end_date, feature_name, poi):
    """
    Retrieve time series data for the specified period.
    """
    def extract_data(image):
        value = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=350).get(feature_name)
        date = image.date().format()
        return ee.Feature(None, {'date': date, feature_name: value})

    time_series = collection.filterDate(start_date, end_date).map(extract_data)

    # Get the data as a list of features
    data_list = time_series.getInfo()['features']
    
    # Initialize lists for dates and values
    dates = []
    values = []

    # Process the list to extract dates and values
    for data in data_list:
        properties = data['properties']
        if feature_name in properties and properties[feature_name] is not None:
            dates.append(properties['date'])
            values.append(properties[feature_name])

    return dates, values

# Define datasets
datasets = {
    'NDVI': ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI'),
    'EVI': ee.ImageCollection('MODIS/006/MOD13Q1').select('EVI'),
    'sur_refl_b01': ee.ImageCollection('MODIS/006/MOD13Q1').select('sur_refl_b01'),
    'sur_refl_b02': ee.ImageCollection('MODIS/006/MOD13Q1').select('sur_refl_b02'),
    'sur_refl_b03': ee.ImageCollection('MODIS/006/MOD13Q1').select('sur_refl_b03'),
    'sur_refl_b07': ee.ImageCollection('MODIS/006/MOD13Q1').select('sur_refl_b07'),

    'dayl': ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('dayl'),
    'prcp': ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('prcp'),
    'srad': ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('srad'),
    'tmax': ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('tmax'),
    'tmin': ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('tmin'),

    'LST_Day_1km': ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Day_1km'),
    'LST_Night_1km': ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Night_1km'),
    'Emis_31': ee.ImageCollection('MODIS/006/MOD11A1').select('Emis_31'),
    'Emis_32': ee.ImageCollection('MODIS/006/MOD11A1').select('Emis_32'),
    'Clear_day_cov': ee.ImageCollection('MODIS/006/MOD11A1').select('Clear_day_cov'),
    'Clear_night_cov': ee.ImageCollection('MODIS/006/MOD11A1').select('Clear_night_cov'),

    'Albedo_BSA_nir': ee.ImageCollection('MODIS/061/MCD43A3').select('Albedo_BSA_nir'),
    'Albedo_BSA_shortwave': ee.ImageCollection('MODIS/061/MCD43A3').select('Albedo_BSA_shortwave'),
    'Albedo_WSA_nir': ee.ImageCollection('MODIS/061/MCD43A3').select('Albedo_WSA_nir'),
    'Albedo_WSA_shortwave': ee.ImageCollection('MODIS/061/MCD43A3').select('Albedo_WSA_shortwave'),

#     'NDSI_Snow_Cover': ee.ImageCollection('MODIS/061/MOD10A1').select('NDSI_Snow_Cover'),
#     'Nadir_Reflectance_Band1': ee.ImageCollection('MODIS/006/MCD43A4').select('Nadir_Reflectance_Band1'),

#     'Fpar': ee.ImageCollection('MODIS/061/MCD15A3H').select('Fpar'),
}

# Load your dataset
# Replace with the path to your dataset file
df = pd.read_csv('Task_FireDetection_noRS_negClass_V4.csv')
df['DetectionDate_first'] = pd.to_datetime(df['DetectionDate_first'], dayfirst=True).dt.strftime('%Y-%m-%d')

# Process each row in the dataset
# Process and save each feature's time series
for feature_name, collection in datasets.items():
    print(f'{feature_name}', end=',')
    all_dates = []
    all_values = []

    for index, row in df.iterrows():
#         print(f'{index}', end=',')
        lat, lon, fire_date_str = row['avgLatitude'], row['avgLongitude'], row['DetectionDate_first']
        fire_date = pd.to_datetime(fire_date_str)
        poi = ee.Geometry.Point(lon, lat)
        start_date = fire_date - pd.DateOffset(days=90)
        end_date = fire_date + pd.DateOffset(days=1)

        # Get time series data for this feature and fire event
        try:
            dates, values = get_time_series(collection, start_date, end_date, feature_name, poi)
        except Exception as e:
            print(f"Error processing location ({lat}, {lon}) for feature {feature_name}: {str(e)}")

        # Add the fire date at the beginning of each list
        dates.insert(0, row['DetectionDate_first'])
        values.insert(0, row['DetectionDate_first'])
#         print(f'{values}', end=',')

        # Append these lists to the respective all_dates and all_values lists
        all_dates.append(dates)
        all_values.append(values)

    # Create DataFrames from the lists of all dates and all values
    dates_df = pd.DataFrame(all_dates)
    values_df = pd.DataFrame(all_values)

    # Save the DataFrames to CSV files
    dates_df.to_csv(f'{feature_name}_negClassV4_dates.csv', index=False)
    values_df.to_csv(f'{feature_name}_negClassV4_values.csv', index=False)

NDVI,EVI,sur_refl_b01,sur_refl_b02,sur_refl_b03,sur_refl_b07,dayl,prcp,srad,tmax,tmin,LST_Day_1km,LST_Night_1km,Emis_31,Clear_day_cov,Clear_night_cov,Albedo_BSA_nir,Albedo_BSA_shortwave,Albedo_WSA_nir,Albedo_WSA_shortwave,

In [10]:
#ELEVATION DATA
import ee
import pandas as pd

# Initialize the Earth Engine module
ee.Initialize()

# Function to get elevation and population
def get_topographical_features(lat, lon):
    try:
        point = ee.Geometry.Point([lon, lat])
        
        # Load the elevation data
        elevation_dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM').mean()

        # Calculate slope and aspect
        slope = ee.Terrain.slope(elevation_dataset)
        aspect = ee.Terrain.aspect(elevation_dataset)

        # Reduce region to get values
        elev_value = elevation_dataset.reduceRegion(ee.Reducer.mean(), point, scale=350).get('DSM').getInfo()
        slope_value = slope.reduceRegion(ee.Reducer.mean(), point, scale=350).get('slope').getInfo()
        aspect_value = aspect.reduceRegion(ee.Reducer.mean(), point, scale=350).get('aspect').getInfo()

        return [
            elev_value if elev_value is not None else pd.NA,
            slope_value if slope_value is not None else pd.NA,
            aspect_value if aspect_value is not None else pd.NA
        ]
    except Exception as e:
        return [pd.NA, pd.NA, pd.NA, str(e)]  # Return NA for all values and the error message

# Load your DataFrame
df = pd.read_csv('Task_FireDetection_noRS_negClass_V4.csv')

# Initialize columns for elevation, slope, and aspect
df['Elevation'] = pd.NA
df['Slope'] = pd.NA
df['Aspect'] = pd.NA

# Iterate over DataFrame rows
for index, row in df.iterrows():
    lat, lon = row['avgLatitude'], row['avgLongitude']
    elev,slo,asp = get_topographical_features(lat, lon)
#     print(elev,slo,asp)
    
    # Store results in DataFrame
    df.at[index, 'Elevation'] = elev
    df.at[index, 'Slope'] = slo
    df.at[index, 'Aspect'] = asp
    
df.to_csv('Elevation_Slope_Aspect_negClassV4_values.csv', index=False)
print('Gathered Elevation, Slope and Aspect!')

Gathered Elevation, Slope and Aspect!


In [11]:
import ee
import pandas as pd

ee.Initialize()  # Initialize the Earth Engine module.

def get_LC(lat, lon, year, lc_type):
    try:
        point = ee.Geometry.Point([lon, lat])
        # Load the MODIS land cover image collection
        LC_collection = ee.ImageCollection("MODIS/061/MCD12Q1").select(lc_type)
        # Filter the collection for the specified year
        LC_image = LC_collection.filter(ee.Filter.calendarRange(year, year, 'year')).first()
        # Reduce the region around the point
        LC_value = LC_image.reduceRegion(
            ee.Reducer.mode(), 
            point.buffer(1000),  # 1km buffer around the point
            scale=100  # Scale in meters; adjust as necessary
        ).get(lc_type).getInfo()

        return LC_value if LC_value is not None else 'No Data'
    except Exception as e:
        return f'Error: {str(e)}'

# Land cover types to process
lc_types = ['LC_Type1','LC_Type2', 'LC_Type3', 'LC_Type4', 'LC_Type5']  # 'LC_Type1', 

# Process each row in the dataset for each land cover type
for lc_type in lc_types:
    # Load your dataset
    df = pd.read_csv('Task_FireDetection_noRS_negClass_V4.csv')
    df['DetectionDate_first'] = pd.to_datetime(df['DetectionDate_first'], dayfirst=True).dt.strftime('%Y-%m-%d')
    df['Year'] = pd.to_datetime(df['DetectionDate_first']).dt.year

    print(f'{lc_type}', end=',')
    for index, row in df.iterrows():
        lat, lon, year = row['avgLatitude'], row['avgLongitude'], row['Year']
        lc_value = get_LC(lat, lon, year, lc_type)

        # Store results in DataFrame
        df.at[index, lc_type] = lc_value

    # Save the DataFrame to a new CSV file
    df.to_csv(f'{lc_type}_negClassV4_values.csv', index=False)

LC_Type1,LC_Type2,LC_Type3,LC_Type4,LC_Type5,

In [ ]:
import ee
import pandas as pd

ee.Initialize()  # Initialize the Earth Engine module.

def get_time_series(collection, start_date, end_date, feature_name, poi):
    """
    Retrieve time series data for the specified period.
    """
    def extract_data(image):
        value = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=350).get(feature_name)
        date = image.date().format()
        return ee.Feature(None, {'date': date, feature_name: value})

    time_series = collection.filterDate(start_date, end_date).map(extract_data)

    # Get the data as a list of features
    data_list = time_series.getInfo()['features']
    
    # Initialize lists for dates and values
    dates = []
    values = []

    # Process the list to extract dates and values
    for data in data_list:
        properties = data['properties']
        if feature_name in properties and properties[feature_name] is not None:
            dates.append(properties['date'])
            values.append(properties[feature_name])

    return dates, values

# Define datasets
datasets = {
#     'v_component_of_wind_10m': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('v_component_of_wind_10m'),
    'u_component_of_wind_10m': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('u_component_of_wind_10m'),
    'surface_pressure': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('surface_pressure'),
    'runoff_sum': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('runoff_sum'),
    'total_evaporation_sum': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('total_evaporation_sum'),
    'snowfall_sum': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('snowfall_sum'),
    'snowmelt_sum': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('snowmelt_sum'),
    'snow_depth': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('snow_depth'),
    'snow_density': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('snow_density'),
    'snow_cover': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('snow_cover'),
    'dewpoint_temperature_2m': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('dewpoint_temperature_2m'),
    'temperature_2m': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('temperature_2m')
}

# Load your dataset
# Replace with the path to your dataset file
df = pd.read_csv('Task_FireDetection_noRS_negClass_V4.csv')
df['DetectionDate_first'] = pd.to_datetime(df['DetectionDate_first'], dayfirst=True).dt.strftime('%Y-%m-%d')

# Process each row in the dataset
# Process and save each feature's time series
for feature_name, collection in datasets.items():
    print(f'{feature_name}', end=',')
    all_dates = []
    all_values = []

    for index, row in df.iterrows():
#         print(f'{index}', end=',')
        lat, lon, fire_date_str = row['avgLatitude'], row['avgLongitude'], row['DetectionDate_first']
        fire_date = pd.to_datetime(fire_date_str)
        poi = ee.Geometry.Point(lon, lat)
        start_date = fire_date - pd.DateOffset(days=90)
        end_date = fire_date + pd.DateOffset(days=1)

        # Get time series data for this feature and fire event
        dates, values = get_time_series(collection, start_date, end_date, feature_name, poi)

        # Add the fire date at the beginning of each list
        dates.insert(0, row['DetectionDate_first'])
        values.insert(0, row['DetectionDate_first'])

        # Append these lists to the respective all_dates and all_values lists
        all_dates.append(dates)
        all_values.append(values)
        
#         print(values)
    # Create DataFrames from the lists of all dates and all values
    dates_df = pd.DataFrame(all_dates)
    values_df = pd.DataFrame(all_values)

    # Save the DataFrames to CSV files
    dates_df.to_csv(f'{feature_name}_negClassV4_dates.csv', index=False)
    values_df.to_csv(f'{feature_name}_negClassV4_values.csv', index=False)